# Simulación de escenarios de reservas y crédito espejo

Este cuaderno tiene las simulaciones para evaluar la diferencia de escenarios de créditos bajo los precios del crédito espejo y los precios de las cosechas de junio 2024. El objetivo es evaluar si la iniciativa del crédito espejo hubiese surtido efecto positivo.

In [21]:
# importar librerías
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from utils import ingresos_financieros

In [22]:
# leer datos
ruta = r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\db\Precios todos iniciativas espejo.xlsx"
liquidez_no_br = pd.read_excel(ruta, sheet_name="LiquidezNoBR")
liquidez_exclusivo = pd.read_excel(ruta, sheet_name="LiquidezExclusivo")
salud_no_br = pd.read_excel(ruta, sheet_name="SaludNoBR")
salud_exclusivo = pd.read_excel(ruta, sheet_name="SaludExclusivo")
datos = pd.read_excel(r"C:\Users\mariajose_chinchilla\Documents\GitHub\Reservas\Salidas\credito_espejo\salidas_Rodrigo\Espejo_junio_cosechas.xlsx")

Ahora que hemos leído los datos, vamos a hacer una simulación con las cosechas de junio. Vamos a ver qué producto fue el que tomó el cliente y como no sabemos si era cliente Banrural o no, o exclusivo Bantrab, vamos a simualr los precios con ambos esquemas.

In [13]:
# # Primero hacemos una limpieza de los datos ajustando el formato a como lo vams a usar
# cols_formatear = ["MONTO_CANCELADO", "MONTO_DESEMBOLSADO_NUEVO"]
# for col in cols_formatear:
#     datos[col] = datos[col].astype(str).str.replace("Q", "")
#     datos[col] = datos[col].astype(str).str.replace(",", "")
#     datos[col] = datos[col].astype("float")


# # cambiar la columna de porcentaje pagado
# datos["%pagado"] = datos["%pagado"].astype(str).str.replace(" %", "")
# datos["%pagado"] = datos["%pagado"].astype("float")

# # cambiar formato a meses para utilizar las formulas ya cradas
# datos["PLAZO_CRED_NUEVO"] = datos["PLAZO_CRED_NUEVO"] / 30

In [14]:
# """ los datos traen una línea por cada crédito cancelado, entonces agrupamos por monto desembolsado
# para trabajr la simulción con esto y las medidas de agrupación necesarias """
# datos_temp = datos.groupby(by="NO_CREDITO_NUEVO")["MONTO_CANCELADO"].sum().reset_index(name="MONTO CANCELADO POR RECREDITO")
# datos = pd.merge(datos, datos_temp, how="left", on="NO_CREDITO_NUEVO", suffixes=("", "_temp"))

## Aplicación de precios

ACá aplicamos los precios establecidos en las iniciativas separando por promoción.

In [23]:
# función para calcular el precio dada una tabla de precios externa de excel
def aplicar_precio(rci: float, riesgo: str, precios: pd.DataFrame) -> float:
    if rci <= 55:
        precio = precios[precios["RCI"] >= rci][riesgo].reset_index(drop=True)[0]
    elif rci > 55 and rci < 60:
        precio = precios[precios["RCI"] == 45][riesgo].reset_index(drop=True)[0]
    return precio

In [24]:
# poner bandera de cumplimiento del 15%
#datos_temp = datos.groupby(by="NO_CREDITO")["%pagado"].min().reset_index(name="MINIMO PORCENTAJE")
#datos = pd.merge(datos, datos_temp, how="left", on="NO_CREDITO_NUEVO", suffixes=("", "_x"))
# datos["FLAG 15%"] = np.where(datos["MINIMO PORCENTAJE"] < 15, 0, 1)
datos["FLAG 15%"] = 0
# nos quedamos solo con los créditos que generaron reservas, es decir, los que no tengan el 15%
mascara_15 = datos["FLAG 15%"] == 0
mascara_liquidez = datos["PRODUCTO"] == "LIQUIDEZ"
mascara_exclusivo = datos["EXCLUSIVO BANTRAB"] = 1
# Aplicar esquemas de precios
datos.loc[mascara_15 & mascara_liquidez, "LIQUIDEZ NO BANRURAL"] = datos.loc[mascara_15 & mascara_liquidez].apply(lambda x: aplicar_precio(x["rciInternoInicialDeudor"], x["scoreDeudorInstallmentQualPacing"], liquidez_no_br), axis=1)
datos.loc[mascara_15 & mascara_liquidez & mascara_exclusivo, "LIQUIDEZ EXCLUSIVO"] = datos.loc[mascara_15 & mascara_liquidez].apply(lambda x: aplicar_precio(x["rciInternoInicialDeudor"], x["scoreDeudorInstallmentQualPacing"], liquidez_exclusivo), axis=1)
datos.loc[mascara_15 & ~mascara_liquidez, "SALUD NO BANRURAL"] = datos.loc[mascara_15 & ~mascara_liquidez].apply(lambda x: aplicar_precio(x["rciInternoInicialDeudor"], x["scoreDeudorInstallmentQualPacing"], salud_no_br), axis=1)
datos.loc[mascara_15 & ~mascara_liquidez & mascara_exclusivo, "SALUD EXCLUSIVO"] = datos.loc[mascara_15 & ~mascara_liquidez].apply(lambda x: aplicar_precio(x["rciInternoInicialDeudor"], x["scoreDeudorInstallmentQualPacing"], salud_exclusivo), axis=1)

## Escoger la data para estadísticos de resumen

In [40]:
df_final = datos[["FECHA_APERTURA", 'NO_CREDITO', 'CODIGO_CLIENTE','TIPO_CLIENTE',
                  'PRODUCTO', "MONTO_DESEMBOLSADO", "DESEMBOLSO_NETO", "TASA_ORIGINAL",
                  "PLAZO_ORIGINAL", "CANTIDAD_DIAS_ATRASO", "dpi",
                  "rciInternoInicialDeudor", "scoreDeudorInstallmentQualPacing",
                  "conSaldoCreditoExt", "EXCLUSIVO BANTRAB", "CREDITOS_VIGENTES",
                  "CATEGORIA", "COTA INFERIOR TASA",  "TASA CVF RESERVAS", 
                  "TASA FINAL", "TASA FINAL NUMERICA", 'LIQUIDEZ NO BANRURAL', 'LIQUIDEZ EXCLUSIVO', 
                  'SALUD NO BANRURAL', 'SALUD EXCLUSIVO', "APLICA RECREDITO"]]

for col in ['LIQUIDEZ NO BANRURAL', 'LIQUIDEZ EXCLUSIVO', 'SALUD NO BANRURAL', 'SALUD EXCLUSIVO']:
    df_final[col].fillna(0, inplace=True)

df_final["CVF MAYOR TASA"] = np.where(df_final["TASA FINAL NUMERICA"]*100 > df_final[['LIQUIDEZ NO BANRURAL', 'LIQUIDEZ EXCLUSIVO', 'SALUD NO BANRURAL', 'SALUD EXCLUSIVO']].max(axis=1),1, 0)

In [51]:
df_final["INGRESOS ORIGINALES"] = df_final.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["TASA_ORIGINAL"]/12, x["PLAZO_ORIGINAL"], 6), axis=1)
df_final["INGRESOS CON MONTO NETO"] = df_final.apply(lambda x: ingresos_financieros(x["DESEMBOLSO_NETO"], x["TASA_ORIGINAL"]/12, x["PLAZO_ORIGINAL"], 6), axis=1)
df_final["INGRESOS CVF BRUTO"] = df_final.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["TASA FINAL NUMERICA"]/12, x["PLAZO_ORIGINAL"], 6), axis=1)
df_final["INGRESOS CVF NETO"] = df_final.apply(lambda x: ingresos_financieros(x["DESEMBOLSO_NETO"], x["TASA FINAL NUMERICA"]/12, x["PLAZO_ORIGINAL"], 6), axis=1)
df_final["INGRESOS LIQUIDEZ EXCLUSIVO"] = df_final.apply(lambda x: ingresos_financieros(x["DESEMBOLSO_NETO"], x["LIQUIDEZ EXCLUSIVO"]/1200, x["PLAZO_ORIGINAL"], 6), axis=1)
df_final["INGRESOS LIQUIDEZ NO BANRURAL"] = df_final.apply(lambda x: ingresos_financieros(x["DESEMBOLSO_NETO"], x["LIQUIDEZ NO BANRURAL"]/1200, x["PLAZO_ORIGINAL"], 6), axis=1)
df_final["INGRESOS SALUD EXCLUSIVO"] = df_final.apply(lambda x: ingresos_financieros(x["DESEMBOLSO_NETO"], x["SALUD EXCLUSIVO"]/1200, x["PLAZO_ORIGINAL"], 6), axis=1)
df_final["INGRESOS SALUD NO BANRURAL"] = df_final.apply(lambda x: ingresos_financieros(x["DESEMBOLSO_NETO"], x["SALUD NO BANRURAL"]/1200, x["PLAZO_ORIGINAL"], 6), axis=1)

In [52]:
bruto = df_final["INGRESOS CVF BRUTO"].sum()
neto = df_final["INGRESOS CVF NETO"].sum()
print(f"BRUTO {bruto} y NETO {neto}")

BRUTO 103945503.13541913 y NETO 29290254.203515176


In [59]:
df_final[df_final["TASA FINAL NUMERICA"] > df_final["TASA_ORIGINAL"]][["COTA INFERIOR TASA", "TASA_ORIGINAL"]]

,COTA INFERIOR TASA,TASA_ORIGINAL
1,0.224979,0.1990
2,0.219379,0.1906
6,0.230979,0.2080
21,0.202312,0.1650
30,0.238979,0.2200
...,...,...
5561,0.216979,0.1870
5563,0.230979,0.2080
5565,0.197646,0.1580
5567,0.202912,0.1659


In [53]:
mascara_no_cvf = df_final["TASA FINAL NUMERICA"] == 0
df_final.loc[mascara_no_cvf, "TASA FINAL NUMERICA"] = (df_final["COTA INFERIOR TASA"] + 0.6 - df_final["rciInternoInicialDeudor"]/100) / 2
df_final["INGRESOS CVF"] = df_final.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["TASA FINAL NUMERICA"]/12, x["PLAZO_ORIGINAL"], 6), axis=1)

In [54]:
df_final.to_excel(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\Salidas\espejo\Retrofit_espejo.xlsx")